<a href="https://colab.research.google.com/github/park-geun-hyeong/Kaggle_project/blob/main/kaggle_sentiment_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
%matplotlib inline

warnings.filterwarnings('ignore')

In [2]:
path="/content/drive/MyDrive/Kaggle/kaggle_bar of words/" #file이 있는 경로 설정

In [3]:
submission=pd.read_csv(path+"sampleSubmission.csv")
submission

,id,sentiment
0,12311_10,0
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0
...,...,...
24995,2155_10,0
24996,59_10,0
24997,2531_1,0
24998,7772_8,0


In [4]:
train=pd.read_csv(path+"labeledTrainData.tsv",delimiter='\t') #csv가 아닌 tsv파일 이기 때문에 읽을 때 delimiter='\t' 추가
test=pd.read_csv(path+"testData.tsv",delimiter='\t')
unlable_train=pd.read_csv(path+"unlabeledTrainData.tsv",delimiter='\t',error_bad_lines=False)

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [5]:
train

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [6]:
test #review를 통해 sentimenter을 예측하는 classifier 문제

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...
...,...,...
24995,2155_10,"Sony Pictures Classics, I'm looking at you! So..."
24996,59_10,I always felt that Ms. Merkerson had never got...
24997,2531_1,I was so disappointed in this movie. I am very...
24998,7772_8,"From the opening sequence, filled with black a..."


In [ ]:
unlable_train #학습을 도와주기 위한 unlabeled file

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."
...,...,...
49993,18984_0,The original Man Eater by Joe D'Amato is somew...
49994,16433_0,When Home Box Office was in it's early days mo...
49995,16006_0,Griffin Dunne was born into a cultural family....
49996,40155_0,"Not a bad story, but the low budget rears its ..."


In [7]:
from bs4 import BeautifulSoup #html을 제거해주기위한
from nltk.corpus import stopwords # 불용어를 위한
from nltk.stem import WordNetLemmatizer #run, ran ,runs -> run으로 바꿔주는 역할
import re #알파벳이아닌 문구를 제거해주기 위한

In [8]:
pip install nltk

In [9]:
import nltk

In [10]:
print(nltk.__version__)

3.2.5


In [11]:
nltk.download('stopwords')
nltk.download('wordnet')

lematizer= WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [12]:
eng_stopwords=stopwords.words('english') #영어 불용어 load
eng_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [13]:
def process_lemma(sentence): #lemmatize를 위한 함수 생성(동사의 여러형태를 원형으로 바꾸어주는 역할)
    return [lematizer.lemmatize(word,'v') for word in sentence]

In [14]:
 def preprocessing(sentence): #review의 전처리를 위한 함수 생성
    soup = BeautifulSoup(sentence,'html.parser') #문장의 html 제거
    clean1 = re.sub('[^a-zA-Z]', ' ' ,soup.text) #문장의 알파벳을 제외한 기호들 제거
    clean2 = clean1.lower() #대문자를 소문자로 변환
    clean3 = [word for word in clean2.split() if word not in eng_stopwords] #불용어 제거
    clean4 = process_lemma(clean3) #동사의 여러형태를 원형으로 변환
    return ' '.join(clean4) #split된 단어들을 문장으로 join하여 return

In [17]:
sample

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [16]:
sample=train['review'][0]
preprocessing(sample) #전처리 전의 sample과 비교해보자.(html제거, 기호제거, 대문자 소문자로, 불용어 제거, 동사원형으로 변환)

'stuff go moment mj start listen music watch odd documentary watch wiz watch moonwalker maybe want get certain insight guy think really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember go see cinema originally release subtle message mj feel towards press also obvious message drug bad kay visually impressive course michael jackson unless remotely like mj anyway go hate find bore may call mj egotist consent make movie mj fan would say make fan true really nice actual feature film bite finally start minutes exclude smooth criminal sequence joe pesci convince psychopathic powerful drug lord want mj dead bad beyond mj overhear plan nah joe pesci character rant want people know supply drug etc dunno maybe hat mj music lot cool things like mj turn car robot whole speed demon sequence also director must patience saint come film kiddy bad sequence usually directors hate work one kid let alone whole bunch perform complex dance scene botto

In [18]:
all_review=pd.concat([train['review'],test['review'],unlable_train['review']])

In [19]:
all_review

0        With all this stuff going down at the moment w...
1        \The Classic War of the Worlds\" by Timothy Hi...
2        The film starts with a manager (Nicholas Bell)...
3        It must be assumed that those who praised this...
4        Superbly trashy and wondrously unpretentious 8...
                               ...                        
49993    The original Man Eater by Joe D'Amato is somew...
49994    When Home Box Office was in it's early days mo...
49995    Griffin Dunne was born into a cultural family....
49996    Not a bad story, but the low budget rears its ...
49997    This not-very-good mummy-alien flick does feat...
Name: review, Length: 99998, dtype: object

In [20]:
print(train.shape)
print(test.shape)
print(unlable_train.shape)
print(all_review.shape)

(25000, 3)
(25000, 2)
(49998, 2)
(99998,)


In [21]:
all_review_cleaned=all_review.apply(preprocessing) #all_review라는 DataFrame에 preprocessing 함수를 적용하여 all_review_cleaned라는 변수에 입력
all_review_cleaned.head()

0    stuff go moment mj start listen music watch od...
1    classic war worlds timothy hines entertain fil...
2    film start manager nicholas bell give welcome ...
3    must assume praise film greatest film opera ev...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [24]:
tokenizer=Tokenizer(oov_token='<OOV>') #oov=out of vocaburary

In [25]:
tokenizer.fit_on_texts(all_review_cleaned) #전처리 되어진 all_review_cleaned를 토큰화 시켜주기

In [32]:
print(len(tokenizer.word_index))
print(tokenizer.word_index)

126312
{'<OOV>': 1, 'film': 2, 'movie': 3, 'one': 4, 'make': 5, 'like': 6, 'see': 7, 'get': 8, 'time': 9, 'good': 10, 'character': 11, 'go': 12, 'watch': 13, 'even': 14, 'would': 15, 'think': 16, 'story': 17, 'really': 18, 'well': 19, 'show': 20, 'look': 21, 'much': 22, 'end': 23, 'know': 24, 'say': 25, 'bad': 26, 'people': 27, 'great': 28, 'also': 29, 'first': 30, 'take': 31, 'give': 32, 'act': 33, 'play': 34, 'love': 35, 'come': 36, 'find': 37, 'way': 38, 'could': 39, 'movies': 40, 'seem': 41, 'plot': 42, 'work': 43, 'two': 44, 'many': 45, 'want': 46, 'never': 47, 'life': 48, 'try': 49, 'best': 50, 'little': 51, 'ever': 52, 'man': 53, 'better': 54, 'scene': 55, 'still': 56, 'scenes': 57, 'part': 58, 'feel': 59, 'something': 60, 'use': 61, 'back': 62, 'interest': 63, 'lot': 64, 'real': 65, 'guy': 66, 'thing': 67, 'director': 68, 'actors': 69, 'funny': 70, 'though': 71, 'cast': 72, 'star': 73, 'years': 74, 'live': 75, 'another': 76, 'old': 77, 'leave': 78, 'actually': 79, 'nothing': 80

In [27]:
train_feature=all_review_cleaned[:len(train)]
test_feature=all_review_cleaned[25000:50000]

In [29]:
print(train_feature.shape)
print(test_feature.shape)
train_label=train['sentiment']
train_label.shape

(25000,)
(25000,)


(25000,)

In [33]:
train_seq = tokenizer.texts_to_sequences(train_feature)
test_seq = tokenizer.texts_to_sequences(test_feature) #texts_to_sequences함수를 통해서 토큰화 되어진 단어들을 sequence형태로 변환시켜주기

In [41]:
train_seq[0]

[397,
 12,
 463,
 11594,
 83,
 931,
 127,
 13,
 895,
 507,
 13,
 21106,
 13,
 19437,
 179,
 46,
 8,
 639,
 2250,
 66,
 16,
 18,
 469,
 3273,
 179,
 5,
 188,
 643,
 2110,
 1155,
 19437,
 58,
 4431,
 58,
 258,
 2,
 240,
 12,
 7,
 349,
 1643,
 255,
 1145,
 550,
 11594,
 59,
 773,
 2039,
 29,
 471,
 550,
 593,
 26,
 4231,
 1924,
 1032,
 175,
 420,
 1453,
 782,
 2209,
 6,
 11594,
 459,
 12,
 613,
 37,
 170,
 116,
 146,
 11594,
 34889,
 9296,
 5,
 3,
 11594,
 109,
 15,
 25,
 5,
 109,
 198,
 18,
 253,
 727,
 258,
 2,
 114,
 339,
 83,
 141,
 7788,
 3475,
 1502,
 311,
 781,
 6909,
 526,
 9123,
 785,
 593,
 1370,
 46,
 11594,
 242,
 26,
 558,
 11594,
 9785,
 505,
 12451,
 781,
 6909,
 11,
 3763,
 46,
 27,
 24,
 2666,
 593,
 413,
 8743,
 179,
 724,
 11594,
 127,
 64,
 469,
 94,
 6,
 11594,
 90,
 419,
 1905,
 130,
 1523,
 2147,
 311,
 29,
 68,
 113,
 3929,
 3388,
 36,
 2,
 22089,
 26,
 311,
 516,
 843,
 613,
 43,
 4,
 129,
 152,
 518,
 130,
 630,
 890,
 1120,
 423,
 55,
 1131,
 107,
 3,
 27,
 6,
 

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [42]:
train_padded = pad_sequences(train_seq, maxlen=200, truncating='post',padding='post')
test_padded = pad_sequences(test_seq, maxlen=200, padding='post', truncating='post') #sequence의 길이가 길거나 짧을때 맞추어주는 역할(truncating=길경우 잘라준다, padding=짧을경우 채워준다.)

In [43]:
print(train_padded.shape)
print(test_padded.shape)

(25000, 200)
(25000, 200)


In [44]:
print(train_padded[0])

[  397    12   463 11594    83   931   127    13   895   507    13 21106
    13 19437   179    46     8   639  2250    66    16    18   469  3273
   179     5   188   643  2110  1155 19437    58  4431    58   258     2
   240    12     7   349  1643   255  1145   550 11594    59   773  2039
    29   471   550   593    26  4231  1924  1032   175   420  1453   782
  2209     6 11594   459    12   613    37   170   116   146 11594 34889
  9296     5     3 11594   109    15    25     5   109   198    18   253
   727   258     2   114   339    83   141  7788  3475  1502   311   781
  6909   526  9123   785   593  1370    46 11594   242    26   558 11594
  9785   505 12451   781  6909    11  3763    46    27    24  2666   593
   413  8743   179   724 11594   127    64   469    94     6 11594    90
   419  1905   130  1523  2147   311    29    68   113  3929  3388    36
     2 22089    26   311   516   843   613    43     4   129   152   518
   130   630   890  1120   423    55  1131   107   

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
train_label=train['sentiment']

In [47]:
x_train,x_valid,y_train,y_valid=train_test_split(train_padded, train_label, stratify=train_label, test_size=0.1,random_state=0)

In [50]:
embedding_dim=300
vocab_size=len(tokenizer.word_index)+1
vocab_size

126313

In [49]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [51]:
#deep learning을 위한 model 구축!

model=Sequential([
     Embedding(vocab_size,embedding_dim, input_length=200), #embedding을통한 차원축소를 통해 overfitting 방지 = review한개는 300개의 토큰으로 되어져있고, 토큰은 300차원으로 embedding되어져 있다.
     Bidirectional(LSTM(128, return_sequences=True)), 
     Bidirectional(LSTM(128)),
     Dropout(0.25),
     Dense(32,activation='relu'),
     Dense(1, activation='sigmoid')

])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 300)          37893900  
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 256)          439296    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                8224      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 38,735,693
Trainable params: 38,735,693
Non-trainable params: 0
____________________________________________

In [52]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [55]:
checkpoint_path='tmp/checkpoint.ckpt'
checkpoint=ModelCheckpoint(filepath=checkpoint_path,save_best_only=True,save_weights_only=True,monitor='val_loss',verbose=1)

In [56]:
model.fit(x_train, y_train, validation_data=(x_valid,y_valid), callbacks=[checkpoint], epochs=5, batch_size=128)

Epoch 1/5
176/176 [==============================] - 717s 4s/step - loss: 0.5243 - acc: 0.7143 - val_loss: 0.3169 - val_acc: 0.8724

Epoch 00001: val_loss improved from inf to 0.31688, saving model to tmp/checkpoint.ckpt
Epoch 2/5
176/176 [==============================] - 721s 4s/step - loss: 0.1855 - acc: 0.9340 - val_loss: 0.3200 - val_acc: 0.8800

Epoch 00002: val_loss did not improve from 0.31688
Epoch 3/5
176/176 [==============================] - 715s 4s/step - loss: 0.0762 - acc: 0.9761 - val_loss: 0.3794 - val_acc: 0.8580

Epoch 00003: val_loss did not improve from 0.31688
Epoch 4/5
 26/176 [===>..........................] - ETA: 9:52 - loss: 0.0446 - acc: 0.9879

KeyboardInterrupt: ignored

In [57]:
model.load_weights(checkpoint_path)

In [58]:
model.evaluate(x_valid,y_valid)

79/79 [==============================] - 28s 355ms/step - loss: 0.3169 - acc: 0.8724


[0.3168767988681793, 0.8723999857902527]

In [59]:
pred = model.predict(test_padded)

In [60]:
pred[pred>=0.5] = 1
pred[pred<0.5] = 0 

In [61]:
pred

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

In [62]:
pred_df=pd.DataFrame(pred)
pred_df

,0
0,1.0
1,0.0
2,0.0
3,1.0
4,1.0
...,...
24995,0.0
24996,1.0
24997,0.0
24998,1.0


In [63]:
submission

,id,sentiment
0,12311_10,0
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0
...,...,...
24995,2155_10,0
24996,59_10,0
24997,2531_1,0
24998,7772_8,0


In [64]:
submission.loc[:,'sentiment']=pred_df[0]
submission

,id,sentiment
0,12311_10,1.0
1,8348_2,0.0
2,5828_4,0.0
3,7186_2,1.0
4,12128_7,1.0
...,...,...
24995,2155_10,0.0
24996,59_10,1.0
24997,2531_1,0.0
24998,7772_8,1.0


In [65]:
submission['sentiment'] = submission['sentiment'].astype('int')
submission

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,0
3,7186_2,1
4,12128_7,1
...,...,...
24995,2155_10,0
24996,59_10,1
24997,2531_1,0
24998,7772_8,1


In [66]:
submission.to_csv("kaggle_nlp4.csv",index=False)